In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="<api-key>")
project = rf.workspace("signlanguagedetection-mcg18").project("sign-language-detection-yot6v")
version = project.version(2)
dataset = version.download("yolov5")

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

In [ ]:
!pip install -qr requirements.txt
import torch
from IPython.display import Image, clear_output
print('Setup Complete')

In [ ]:
%cd ..
%cat dataset/data.yaml

In [ ]:
import yaml
with open('/content/dataset/data.yaml','r') as file:
  num_classes = str(yaml.safe_load(file)['nc'])

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def write_template(line,cell):
  with open(line,'w') as f:
    f.write(cell.format(**globals()))

In [ ]:
%%write_template /content/yolov5/models/custom_yolov5s.yaml

nc: {num_classes}
depth_multiple: 0.33
width_multiple: 0.50

anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 300 --data '../dataset/data.yaml' --cfg ./models/custom_yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results  --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
from utils.plots import plot_results
Image(filename='runs/train/yolov5s_results3/results.png',width=1000)

In [ ]:
from utils.plots import plot_results
Image(filename='runs/train/yolov5s_results3/val_batch0_pred.jpg',width=1000)